## Trabajo Práctico 2: Redes Neuronales

In [1]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import keras
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
!nvidia-smi

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import keras
import tensorflow as tf

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

RANDOM_STATE = 19 * 103785

GSPREADHSEET_DOWNLOAD_URL = ("https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format)

DF_TRAIN_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
DF_HOLDOUT_GID = "1ObsojtXfzvwicsFieGINPx500oGbUoaVTERTc69pzxE"

def tiene_n_missings(x, n):
    acum = 0
    for i in range(len(x)):
        if x[i]:
            acum += 1
    return n <= acum


def feature_engineering(df):
    lista_de_missings = [tiene_n_missings(x, 3) for x in df.isna().to_numpy()]
    df_3_missings = df.reset_index()[lista_de_missings].set_index('index')
    df.drop(df_3_missings.index, inplace = True)
    df = df.replace({'trabajo': np.nan, 'categoria_de_trabajo': np.nan},'No contesto')
    df = df.replace({'barrio': np.nan},'Palermo')
    mapa = {'Palermo' : 'C14','Belgrano' : 'C13','San Isidro' : 'S.I','Villa Urquiza' : 'C12','Recoleta' : 'C2','La Boca' : 'C4','Agronomia' : 'C15','Almagro' : 'C5','Balvanera' : 'C3','Puerto Madero' : 'C1','Caballito' : 'C6','Boedo' : 'C5','Barracas' : 'C4','Chacarita' : 'C15','Coghland' : 'C12','Floresta' : 'C10','Constitucion' : 'C1','Colegiales' : 'C13','Flores' : 'C7','Liniers' : 'C9','Monte Castro' : 'C10','Mataderos' : 'C9','Nueva Pompeya' : 'C4','Monserrat' : 'C1','nuñez' : 'C13','Parque Chacabuco' : 'C7','Parque Avellaneda' : 'C9','Villa Luro' : 'C10','Parque Chas' : 'C15','La Paternal' : 'C15','Retiro' : 'C1','Villa Devoto' : 'C11','Villa Soldati' : 'C8','San Telmo' : 'C1','Villa Real' : 'C10','Santa Rita' : 'C11','Villa General Mitre' : 'C11','Versalles' : 'C10','Velez Sarsfield' : 'C10','Villa Pueyrredon' : 'C12','Cilla Riachuelo' : 'C8'}
    df['comuna'] = df['barrio'].apply(lambda x: mapa.get(x))
    df_gente_sin_trabajo_con_horas_registradas = df[(df['categoria_de_trabajo'] == 'sin_trabajo') & (df['horas_trabajo_registradas'] > 0)]
    df.drop(df_gente_sin_trabajo_con_horas_registradas.index, inplace = True)
    df['rol_familiar_registrado'] = df['rol_familiar_registrado'].apply(lambda x: 'casado' if x == 'casada' else x)
    df['estado_marital'] = df['estado_marital'].apply(lambda x: 'matrimonio' if x == 'matrimonio_civil' or x == 'matrimonio_militar' else x)
    df = df.drop(['educacion_alcanzada'],axis=1)
    return df


def obtenerDF(GID):
    return pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=GID), skiprows=0)

def obtenerDFTraining():
    return obtenerDF(DF_TRAIN_GID)

def obtenerDFHoldout():
    return obtenerDF(DF_HOLDOUT_GID)

def preprocesar_df_min_max_scaler(X : pd.DataFrame):
    X = pd.get_dummies(X)
    scaler = MinMaxScaler()
    scaler.fit(X)
    return scaler.transform(X)

def preprocesar_df_pca(df , dim):
    Y = df['tiene_alto_valor_adquisitivo']
    X = df.drop(columns=['tiene_alto_valor_adquisitivo'])
    X = preprocesar_df_min_max_scaler(X)
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=RANDOM_STATE)
    
    pca = PCA(dim)
    x_train_pca = pd.DataFrame(pca.fit_transform(x_train))
    x_test_pca = pca.transform(x_test)
    
    return (x_train_pca,x_test_pca,y_train,y_test)

def preprocesar_data_frame(df : pd.DataFrame):
    df = feature_engineering(df)
    y = df['tiene_alto_valor_adquisitivo']
    X = df.drop(columns=['tiene_alto_valor_adquisitivo'])
    return (X, y)


def prepros_dummies(data):
    
    data_prepos = pd.get_dummies(data[['ganancia_perdida_declarada_bolsa_argentina','trabajo',
                                      'estado_marital','edad','genero',"anios_estudiados","horas_trabajo_registradas"]])
    
    data_prepos['ganancia_perdida_declarada_bolsa_argentina'] = data_prepos['ganancia_perdida_declarada_bolsa_argentina'].apply(lambda x: np.tanh(x))
    data_prepos = data_prepos.drop(['trabajo_No contesto'],axis = 1)
    data_prepos = data_prepos.drop(['genero_mujer'],axis = 1)
    data_prepos = data_prepos.drop(['estado_marital_divorciado'],axis = 1)

    return data_prepos



df = obtenerDFTraining()
(X,y) = preprocesar_data_frame(df)
X_prepos= prepros_dummies(X)

x_train, x_test, y_train, y_test = train_test_split(X_prepos, y, test_size=0.25, random_state=RANDOM_STATE)
#x_train, x_test, y_train, y_test  = preprocesar_df_pca(df[['ganancia_perdida_declarada_bolsa_argentina','trabajo',
#                                      'estado_marital',"anios_estudiados",'genero',"tiene_alto_valor_adquisitivo"]],18)

#x_train, x_test, y_train, y_test  = preprocesar_df_pca(df,30)
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(24,)))
model.add(Dropout(0.5))
model.add(Dense(12, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
opt = keras.optimizers.Adam()
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()

h = model.fit(x_train, y_train, epochs=1000,batch_size=1000, verbose=0, validation_split=0.3)

plt.figure(dpi=125, figsize=(12, 4))
plt.plot(h.history['loss'], label="loss")
plt.plot(h.history['val_loss'], label="validation loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

plt.figure(dpi=125, figsize=(12, 6))
plt.plot(h.history['accuracy'], label="accuracy")
plt.plot(h.history['val_accuracy'], label="validation accuracy")
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,h.model.predict(x_test,verbose =4 )>0.5))
print(classification_report(y_test,h.model.predict(x_test,verbose =4 )>0.45))
print(classification_report(y_test,h.model.predict(x_test,verbose =4 )>0.40))

h.model.predict(x_test,verbose =4 )